## Evaluation of Models

In [4]:
import numpy as np
import pandas as pd

In [ ]:
forecast_master=pd.read_csv('C:/Users/Admin/NIYOJAN/Forecasts/forecast_master.csv')

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [8]:
results = (
    forecast_master
    .groupby(["Product", "Model"])
    .apply(lambda g: pd.Series({
        "MAPE": mape(g["Actual"], g["Forecast"]),
        "RMSE": rmse(g["Actual"], g["Forecast"]),
        "MAE": mae(g["Actual"], g["Forecast"])
    }))
    .reset_index()
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15252\1841825358.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [13]:
results.sample(10)

,Product,Model,MAPE,RMSE,MAE
15,Sunscreen,XGBoost,21.208077,4.750439,3.633333
16,Travel_Antidiarrheal,XGBoost,21.236296,4.813176,3.633333
6,Insulin_Supplies,LSTM,16.378477,12.404423,10.068242
2,Antibiotic_A,XGBoost,14.944504,5.507571,4.533333
9,ORS_Solution,XGBoost,11.869063,8.951722,7.533333
3,Antihistamine,Prophet,16.270847,3.612257,2.900784
5,Cough_Syrup,Prophet,10.813428,14.257021,7.073845
12,Paracetamol_500mg,LSTM,13.311868,12.313900,10.162996
4,Cough_Syrup,ARIMA,9.040577,2.758840,1.728790
17,Vitamin_C,ARIMA,5.664047,3.297656,2.237978


In [15]:
results.to_csv("C:/Users/Admin/IIS/Evaluation/model_evaluation_results.csv", index=False)

## Selecting Best Model

In [17]:
# Select the row with lowest RMSE for each product
best_models = (
    results.loc[results.groupby("Product")["RMSE"].idxmin()]
    [["Product", "Model", "RMSE"]]
    .rename(columns={"Model": "Best_Model", "RMSE": "Error_Score"})
    .reset_index(drop=True)
)

In [21]:
best_models.head(10)

,Product,Best_Model,Error_Score
0,Antibiotic_A,ARIMA,2.259436
1,Antihistamine,Prophet,3.612257
2,Cough_Syrup,ARIMA,2.758840
3,Insulin_Supplies,LSTM,12.404423
4,ORS_Solution,ARIMA,2.514885
5,PainRelief_Gel,LSTM,19.670599
6,Paracetamol_500mg,ARIMA,7.618366
7,Sunscreen,Prophet,3.438714
8,Travel_Antidiarrheal,XGBoost,4.813176
9,Vitamin_C,ARIMA,3.297656


In [ ]:
best_models.to_csv("C:/Users/Admin/NIYOJAN/Evaluation/best_models_per_product.csv", index=False)

## Standardize Forecast Outputs

In [23]:
# Merge master forecasts with best model table
final_forecasts = forecast_master.merge(
    best_models,
    left_on=["Product", "Model"],
    right_on=["Product", "Best_Model"],
    how="inner"
)

In [24]:
# Keep only required columns
final_forecasts = final_forecasts[["Date", "Product", "Forecast"]].rename(
    columns={"Forecast": "Forecast_Sales"}
)

In [25]:
# Sort for neatness
final_forecasts = final_forecasts.sort_values(["Product", "Date"]).reset_index(drop=True)

In [38]:
final_forecasts.sample(10)

,Date,Product,Forecast_Sales
58,2024-12-29,Antihistamine,23.261299
71,2024-12-12,Cough_Syrup,23.044094
89,2024-12-30,Cough_Syrup,19.707716
220,2024-12-11,Sunscreen,13.388565
92,2024-12-03,Insulin_Supplies,59.251700
267,2024-12-28,Travel_Antidiarrheal,14.000000
190,2024-12-11,Paracetamol_500mg,60.232085
144,2024-12-25,ORS_Solution,8.800441
128,2024-12-09,ORS_Solution,8.074105
246,2024-12-07,Travel_Antidiarrheal,18.000000


In [39]:
final_forecasts.shape

(300, 3)

In [ ]:
final_forecasts.to_csv("C:/Users/Admin/NIYOJAN/Forecasts/final_standardized_forecasts.csv", index=False)